<h1 align=center> Tarea 8: Reducción de Dimensionalidad
 </h1>

Integrantes:
- Axel Mondaca
- Sebastián Hernández

In [42]:
%pip install umap-learn
%pip install -U spacy

import spacy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE
import umap.umap_ as umap

from sklearn.impute import SimpleImputer

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [43]:
df = pd.read_csv('customers_Tarea8.csv')
df

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,04-09-2012,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,08-03-2014,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,10-02-2014,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173,...,5,0,0,0,0,0,0,3,11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,10870,1967,Graduation,Married,61223.0,0,1,13-06-2013,46,709,...,5,0,0,0,0,0,0,3,11,0
2236,4001,1946,PhD,Together,64014.0,2,1,10-06-2014,56,406,...,7,0,0,0,1,0,0,3,11,0
2237,7270,1981,Graduation,Divorced,56981.0,0,0,25-01-2014,91,908,...,6,0,1,0,0,0,0,3,11,0
2238,8235,1956,Master,Together,69245.0,0,1,24-01-2014,8,428,...,3,0,0,0,0,0,0,3,11,0


In [44]:
X = df.drop(axis=1, columns=["Response"])
y = df['Response']


In [45]:
numerical = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical = X.select_dtypes(include=['object', 'category']).columns.tolist()

preproc = ColumnTransformer(
    [("num", Pipeline([("imputer", SimpleImputer(strategy='mean')), ("scaler", StandardScaler())]), numerical),
     ("cat", Pipeline([("imputer", SimpleImputer(strategy='most_frequent')), ("encoder", OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=-1))]), categorical)]
)

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=254, stratify=y)

### PCA, Principal Component Analysis

In [47]:
#Sin reduccion
rf_pipe = Pipeline([
    ("preproc", preproc),
    ("clasificador",RandomForestClassifier(random_state=254))])

rf_pipe.fit(X_train, y_train)
y_pred = rf_pipe.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.90      0.96      0.93       381
           1       0.63      0.36      0.46        67

    accuracy                           0.87       448
   macro avg       0.76      0.66      0.69       448
weighted avg       0.86      0.87      0.86       448



In [48]:
#Con reduccion de dimensionalidad
rf_pipe_pca = Pipeline([
    
    ("preproc", preproc),
    ("red_dim", PCA(n_components=5, random_state=254)),
    ("clasificador",RandomForestClassifier(random_state=254))])

rf_pipe_pca.fit(X_train, y_train)
y_pred_pca = rf_pipe_pca.predict(X_test)
print(classification_report(y_test, y_pred_pca))

              precision    recall  f1-score   support

           0       0.89      0.91      0.90       381
           1       0.41      0.34      0.37        67

    accuracy                           0.83       448
   macro avg       0.65      0.63      0.64       448
weighted avg       0.82      0.83      0.82       448



In [53]:
pca = rf_pipe_pca.named_steps['red_dim']
cant_componentes = pca.n_components_
print(f"Cantidad de componentes: {cant_componentes} y varianza explicada: {np.cumsum(pca.explained_variance_ratio_)[-1]*100:.3f}%")

Cantidad de componentes: 5 y varianza explicada: 99.958%
